# Automated Feature Engineering with Featuretools

[Featuretools](https://www.featuretools.com/) is a fantastic python package for automated feature engineering.  It can automatically generate features from secondary datasets which can then be used in machine learning models.  In this post we'll see how automated feature engineering with Featuretools works, and how to run it on complex multi-table datsets!

**Outline**
- [Automated Feature Engineering](#automated-feature-engineering)
- [Deep Feature Synthesis](#deep-feature-synthesis)
- [Using Featuretools](#using-featuretools)
- [Predictions from Generated Features](#predictions-from-generated-features)
- [Running out of Memory](#running-out-of-memory)


<a class="anchor" id="automated-feature-engineering"></a>
## Automated Feature Engineering

What do I mean by "automated feature engineering" and how is it useful?  When building predictive models, we need to have training examples which have some set of features.  For most machine learning algorithms (though of course not all of them), this training set needs to take the form of a table or matrix, where each row corresponds to a single training example or observation, and each column corresponds to a different feature.  For example, suppose we're trying to predict how likely loan applicants are to successfully repay their loans.  In this case, our data table will have a row for each applicant, and a column for each "feature" of the applicants, such as their income, their current level of credit, their age, etc.

Unfortunately, in most applications the data isn't quite as simple as just one table.  We'll likely have additional data stored in other tables!  To continue with the loan repayment prediction example, we could have a separate table which stores the monthly balances of applicants on their other loans, and another separate table with the credit card accounts for each applicant, and yet another table with the credit card activity for each of those accounts, and so on. 

![Data table tree](/assets/img/featuretools/DataframeTree.svg)

In order to build a predictive model, we need to "engineer" features from data in those secondary tables.  These engineered features can then be added to our main data table, which we can then use to train the predictive model.  For example, we could compute the number of credit card accounts for each applicant, and add that as a feature to our primary data table; we could compute the balance across each applicant's credit cards, and add that to the primary data table; we could also compute the balance to available credit ratio and add that as a feature; etc.

With complicated (read: real-life) datasets, the number of features that we could engineer becomes very large, and the task of manually engineering all these features becomes extremely time-intensive.  The [Featuretoools](https://www.featuretools.com/) package automates this process by automatically generating features for our primary data table from information in secondary data sources.

<a class="anchor" id="deep-feature-synthesis"></a>
## Deep Feature Synthesis

**TODO**: explain deep feature synthesis, feature primitives, etc

<a class="anchor" id="using-featuretools"></a>
## Using Featuretools

**TODO**: show how to use it after loading data in w/ pandas, and then run example model on it (e.g. lightGBM)

To show how Featuretools works, we'll be using it on the [Home Credit Group Default Risk](https://www.kaggle.com/c/home-credit-default-risk/data) dataset.  This dataset contains information about individuals applying for loans with [Home Credit Group](http://www.homecredit.net/), a consumer lender specializing in loans to individuals with little credit history.  Home Credit Group hopes to be able to predict how likely an applicant is to default on their loan, in order to decide whether a given loan plan is good for a specific applicant (or whether to suggest a different payment schedule).  

The dataset contains multiple tables which relate to one another in some way.  Below is a diagram which shows each data table, the information it contains, and how each table is related to each other table.

![File connection columns](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

The primary tables (`application_train.csv` and `application_test.csv`) have information on each of the loan applications, where each row corresponds to a single application.  The train table has information about whether that applicant ended up defaulting on their loan, while the test table does not (because those are the applications we'll be testing our predictive model on).  The other tables contain information about other loans (either at other institutions, in the `bureau.csv` and `bureau_balance.csv` tables, or previous loans with Home Credit, in `previous_applications.csv`, `POS_CASH_balance.csv`, `instalments_payments.csv`, and `credit_card_balance.csv`).

What are the relationships between these tables?  The value in the `SK_ID_CURR` column of the `application_*.csv` and `bureau.csv` tables identify the applicant.  That is, to combine the two tables into a single table, we could merge on `SK_ID_CURR`.   Similarly, the `SK_ID_BUREAU` column in `bureau.csv` and `bureau_balance.csv` identifies the applicant, though in this case there can be multiple entries in `bureau_balance.csv` for a single applicant.  The text in the line connecting the tables in the diagram above shows what column two tables can be merged on.

We could manually go through all these databases and construct features based on them, but this would entail not just a lot of manual work, but a *lot* of design decisions.  For example, should we construct a feature which corresponds to the maximum amount of credit the applicant has ever carried?  Or the average amount of credit?  Or the monthly median credit?  Should we construct a feature for how many payments the applicant has made, or how regular their payments are, or *when* they make their payments, etc, etc, etc? 

Featuretools allows us to define our datasets, the relationships between our datasets, and automatically extracts features from child datasets into parent datasets using deep feature synthesis.  We'll use Featuretools to generate features from the data in the secondary tables in the Home Credit Group dataset, and keep features which are informative. 

First let's load the packages we need.

In [3]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from lightgbm import LGBMClassifier
import featuretools as ft
from featuretools import selection

We'll use pandas to load the data.

In [5]:
# Load applications data
Nrows = 10000
train = pd.read_csv('../input/application_train.csv', nrows=Nrows)
test = pd.read_csv('../input/application_test.csv', nrows=Nrows)
bureau = pd.read_csv('../input/bureau.csv', nrows=Nrows)
bureau_balance = pd.read_csv('../input/bureau_balance.csv', nrows=Nrows)
cash_balance = pd.read_csv('../input/POS_CASH_balance.csv', nrows=Nrows)
card_balance = pd.read_csv('../input/credit_card_balance.csv', nrows=Nrows)
prev_app = pd.read_csv('../input/previous_application.csv', nrows=Nrows)
payments = pd.read_csv('../input/installments_payments.csv', nrows=Nrows)

To ensure that featuretools creates the same features for the test set as for the training set, we'll merge the two tables, but add a column which indicates whether each row is a test or training 

In [8]:
# Merge application data
train['Test'] = False
test['Test'] = True
test['TARGET'] = np.nan
app = train.append(test, ignore_index=True, sort=False)

Now we can take a look at the data in the main table.

In [11]:
app.sample(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,Test
9371,110897,0.0,Cash loans,F,N,Y,0,157500.0,755190.0,36459.0,675000.0,Family,Commercial associate,Higher education,Married,House / apartment,0.009657,-11106,-935,-2499.0,-2376,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,TUESDAY,14,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-741.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,False
15127,137286,NaN,Cash loans,M,N,Y,0,157500.0,296280.0,23539.5,225000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.024610,-10642,-1527,-4286.0,-2740,NaN,1,1,1,1,0,0,Laborers,1.0,2,2,SATURDAY,8,0,0,0,0,1,1,...,0.0313,NaN,0.0000,NaN,block of flats,0.0242,"Stone, brick",No,1.0,0.0,1.0,0.0,-516.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0,True
19734,170707,NaN,Cash loans,F,Y,N,1,180000.0,632664.0,35325.0,540000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018029,-14875,-2943,-3397.0,-3671,14.0,1,1,0,1,1,0,Laborers,2.0,3,3,THURSDAY,5,0,0,0,0,0,0,...,0.2145,NaN,0.0000,NaN,block of flats,0.1657,"Stone, brick",No,0.0,0.0,0.0,0.0,-1610.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,True
8556,109961,0.0,Cash loans,F,N,Y,3,135000.0,208854.0,22288.5,184500.0,Family,Working,Secondary / secondary special,Married,House / apartment,0.035792,-13462,-1085,-2303.0,-2307,NaN,1,1,0,1,0,0,Sales staff,5.0,2,2,TUESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False
806,100923,0.0,Cash loans,F,Y,Y,2,270000.0,284256.0,30289.5,270000.0,Family,Commercial associate,Secondary / secondary special,Married,House / apartment,0.007114,-12333,-502,-2571.0,-3232,1.0,1,1,0,1,1,0,Cleaning staff,4.0,2,2,THURSDAY,10,0,0,0,0,0,0,...,0.0423,NaN,0.0305,NaN,block of flats,0.0392,"Stone, brick",No,6.0,0.0,6.0,0.0,-911.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,3.0,False
17873,157859,NaN,Cash loans,F,Y,Y,1,270000.0,1096020.0,56092.5,900000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.011657,-12583,-1231,-6400.0,-2250,10.0,1,1,0,1,0,1,Managers,3.0,1,1,SATURDAY,17,0,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-661.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,True
18679,163386,NaN,Cash loans,M,N,N,0,135000.0,675000.0,25447.5,675000.0,Family,State servant,Secondary / secondary special,Married,With parents,0.018029,-20274,-1629,-2516.0,-2775,NaN,1,1,1,1,0,0,Dr

In [ ]:
# Print info about each column in the dataset
for col in app:
    print(col)
    Nnan = train[col].isnull().sum()
    print('Number empty: ', Nnan)
    print('Percent empty: ', 100*Nnan/train.shape[0])
    print(train[col].describe())
    if train[col].dtype==object:
        print('Categories and Count:')
        print(train[col].value_counts().to_string(header=None))
    print()

In [ ]:
for col in test:
    if test[col].dtype==object:
        print(col)
        print('Num Unique in Train:', train[col].nunique())
        print('Num Unique in Test: ', test[col].nunique())
        print('Unique in Train:', sorted([str(e) for e in train[col].unique().tolist()]))
        print('Unique in Test: ', sorted([str(e) for e in test[col].unique().tolist()]))
        print()

The first step in using Featuretools is to define the "entities", each of which is one data file or table, and the columns along which they are indexed.

In [ ]:
# Define entities
# Each entry is "Name", (df, "id_col_name")
entities = { #use id_col_name not in df for new index, w/ None uses 1st col
    'app': (app, 'SK_ID_CURR'),
    'bureau': (bureau, 'SK_ID_BUREAU'),
    'bureau_balance': (bureau_balance, 'New'),
    'cash_balance': (cash_balance, 'New'),
    'card_balance': (card_balance, 'New'),
    'prev_app': (prev_app, 'SK_ID_PREV'),
    'payments': (payments, 'New') 
}

Next we'll define the relationships between these entities (how each row in one dataset relates to a row in another dataset, based of an ID value in a given column for each entity).

In [ ]:
# Define relationships between dataframes
# Each entry is (parent_entity, parent_variable, child_entity, child_variable)
relationships = [
    ('app', 'SK_ID_CURR', 'bureau', 'SK_ID_CURR'),
    ('bureau', 'SK_ID_BUREAU', 'bureau_balance', 'SK_ID_BUREAU'),
    ('app', 'SK_ID_CURR', 'prev_app', 'SK_ID_CURR'),
    ('app', 'SK_ID_CURR', 'cash_balance', 'SK_ID_CURR'),
    ('app', 'SK_ID_CURR', 'payments', 'SK_ID_CURR'),
    ('app', 'SK_ID_CURR', 'card_balance', 'SK_ID_CURR')
]

Then we'll define which "feature primitives" we want to use to construct features.  First let's look at a list of all the feature primitives available in Featuretools:

In [12]:
pd.options.display.max_rows = 100
ft.list_primitives()

,name,type,description
0,trend,aggregation,Calculates the slope of the linear trend of va...
1,sum,aggregation,Counts the number of elements of a numeric or ...
2,median,aggregation,Finds the median value of any feature with wel...
3,time_since_last,aggregation,Time since last related instance.
4,count,aggregation,Counts the number of non null values.
5,avg_time_between,aggregation,Computes the average time between consecutive ...
6,percent_true,aggregation,Finds the percent of 'True' values in a boolea...
7,all,aggregation,Test if all values are 'True'.
8,n_most_common,aggregation,Finds the N most common elements in a categori...
9,num_true,aggregation,Finds the number of 'True' values in a boolean.


We'll use a simple set of feature primitives: just the mean and the count of entries in the secondary data files.

In [ ]:
# Define which primitives to use
agg_primitives =  ['count', 'mean', 'num_unique', 'percent_true']
trans_primitives = ['time_since_previous']

Finally, we can run deep feature synthesis on our entities given their relationships and a list of feature primitives.

In [ ]:
# Run deep feature synthesis
t0 = time.time()
dfs_feat, dfs_defs = ft.dfs(entities=entities,
                            relationships=relationships,
                            target_entity='app',
                            trans_primitives=trans_primitives,
                            agg_primitives=agg_primitives, 
                            verbose = True,
                            max_depth=2, n_jobs=2)
print('DFS took %0.3g sec' % (time.time()-t0))

In [ ]:
# Delete things we don't need anymore
gc.enable()
del train, test, app, bureau, bureau_balance, cash_balance, prev_app, payments
gc.collect()

# Use featuretools features
app = dfs_feat

If we take a look at the dataframe which was returned by Featuretools, we can see that a bunch of features were appended which correspond to our selected feature primitive functions applied to data in the secondary data files which correspond to each row in the main application dataset.

In [ ]:
app

Now that we've generated a bunch of features, we should make sure to remove ones which don't carry any information.  Featuretools includes a function to remove features which are entirely NULLs or only have one class, etc:

In [ ]:
# Remove low information features
Nf0 = app.shape[1] #number of initial features
app = selection.remove_low_information_features(dfs_feat)
print('Removed', Nf0-app.shape[1], 'features')

In some cases it might also be a good idea to do further feature selection at this point, by, say, removing features which have low mutual information with the target variable (loan default).

<a class="anchor" id="predictions-from-generated-features"></a>
## Predictions from Generated Features

Now that we've generated features using Featuretools, we can use those generated features in a predictive model.  First we have to split our features back into training and test datasets, and remove the indicator columns.

In [ ]:
# Split data back into test + train
train = app.loc[~app['Test'], :]
test = app.loc[app['Test'], :]

# Make SK_ID_CURR the index
train.set_index('SK_ID_CURR', inplace=True)
test.set_index('SK_ID_CURR', inplace=True)

# Ensure all data is stored as floats
train = train.astype(np.float32)
test = test.astype(np.float32)

# Target labels
train_y = train['TARGET']

# Remove test/train indicator column and target column
train.drop(columns=['Test', 'TARGET'], inplace=True)
test.drop(columns=['Test', 'TARGET'], inplace=True)

Then we can run a predictive model, such as LightGBM, on the generated features to predict how likely applicants are to default on their loans.

In [ ]:
# Classification pipeline w/ LightGBM
lgbm_pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('imputer', SimpleImputer(strategy='median')),
    ('classifier', CalibratedClassifierCV(
                        base_estimator=LGBMClassifier(),
                        method='isotonic'))
])

# Fit to training data
lgbm_fit = lgbm_pipeline.fit(train, train_y)

# Predict loan default probabilities of test data
test_pred = lgbm_fit.predict_proba(test)

# Save predictions to file
df_out = pd.DataFrame()
df_out['SK_ID_CURR'] = test.index
df_out['TARGET'] = test_pred[:,1]
df_out.to_csv('test_predictions.csv', index=False)

<a class="anchor" id="running-out-of-memory"></a>
## Running out of Memory

The downside of Featuretools is that is isn't generating features all that intelligently - it simply generates features by applying all the feature primitives to all the features in secondary datasets recursively.  This means that the number of features which are generated can be *huge*!  When dealing with large datasets, this means that the feature generation process might take up more memory than is available on a personal computer.  If you run out of memory, you can always [run featuretools on an Amazon Web Services EC2 instance] (https://brendanhasz.github.io/2018/08/30/aws.html) which has enough memory, such as the `r5` class of instances.